# Visualize suspected leukemia cells

#### This notebook labels each cell within this patient's images by it's Phenograph cluster number generated in Phenograph-CD45_features.ipynb. Suspected Lk cells are outlined in red to visually confirm.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile
import cv2
from PIL import Image
from PIL import ImageDraw
from skimage import data, color, io, img_as_float
from os import listdir
from os.path import isfile, join
import skimage.io
import skimage.util
from skimage import data, segmentation, util, measure
from skimage.segmentation import expand_labels

In [26]:
# create dictionary of directories for each patient
img_dirs = {'P51':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P51_R1158_S1_Normalized_DAPI_B2M_CD34/', 
            'P52':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P52_R1149_S1_Normalized_DAPI_B2M_CD34/', 
            'P53':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P53_R1149_S2_Normalized_DAPI_B2M_CD34/', 
            'P56':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P56_R1158_S2_Normalized_DAPI_B2M_CD34/', 
            'P57':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P57_R1158_S3_Normalized_DAPI_B2M_CD34/', 
            'P58':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P58_R1149_S3_Normalized_DAPI_B2M_CD34/'}

In [53]:
patient = 'P57'

In [54]:
# create dictionary of directories for each patient
mask_dir = '/data/Zhaolab/1_AMLCosMx/Final_scripts/2_Segmentation/3_NucMemMerging/' + patient + '_hybrid/labels_predicted_2_15_24/'

In [55]:
# load in celltype predictions for all cells
morph_predicted = pd.read_csv(patient + '_CD34_subclusters.csv')
morph_predicted

,Unnamed: 0,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters
0,P57_FOV01_cell_10,1,10,A,2559.0,9460.777778,1.0,2,0
1,P57_FOV01_cell_11,1,11,A,990.0,3695.222222,1.0,2,3
2,P57_FOV01_cell_44,1,44,A,619.0,10436.333333,1.0,2,0
3,P57_FOV01_cell_65,1,65,A,2655.0,6428.111111,1.0,2,9
4,P57_FOV01_cell_141,1,141,A,1296.0,1816.166667,1.0,2,7
...,...,...,...,...,...,...,...,...,...
3743,P57_FOV21_cell_2086,21,2086,C,879.0,1048.000000,1.0,2,13
3744,P57_FOV21_cell_2099,21,2099,C,NaN,NaN,1.0,2,13
3745,P57_FOV21_cell_2976,21,2976,C,2654.0,1141.722222,1.0,2,13
3746,P57_FOV21_cell_3256,21,3256,C,1309.0,1453.777778,1.0,2,8


In [56]:
morph_predicted = morph_predicted.drop(columns=["Unnamed: 0"])
morph_predicted

,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters
0,1,10,A,2559.0,9460.777778,1.0,2,0
1,1,11,A,990.0,3695.222222,1.0,2,3
2,1,44,A,619.0,10436.333333,1.0,2,0
3,1,65,A,2655.0,6428.111111,1.0,2,9
4,1,141,A,1296.0,1816.166667,1.0,2,7
...,...,...,...,...,...,...,...,...
3743,21,2086,C,879.0,1048.000000,1.0,2,13
3744,21,2099,C,NaN,NaN,1.0,2,13
3745,21,2976,C,2654.0,1141.722222,1.0,2,13
3746,21,3256,C,1309.0,1453.777778,1.0,2,8


In [57]:
# add col to indicate if cell is predicted CD34+ lk cell
morph_predicted['Lk_cell'] = 0
morph_predicted['Lk_cell'][morph_predicted['PhenoGraph_clusters'].isin([0,3,4,5,8,9,10,11,12])] = 1
morph_predicted

/tmp/ipykernel_2140715/824919165.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morph_predicted['Lk_cell'][morph_predicted['PhenoGraph_clusters'].isin([0,3,4,5,8,9,10,11,12])] = 1


,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters,Lk_cell
0,1,10,A,2559.0,9460.777778,1.0,2,0,1
1,1,11,A,990.0,3695.222222,1.0,2,3,1
2,1,44,A,619.0,10436.333333,1.0,2,0,1
3,1,65,A,2655.0,6428.111111,1.0,2,9,1
4,1,141,A,1296.0,1816.166667,1.0,2,7,0
...,...,...,...,...,...,...,...,...,...
3743,21,2086,C,879.0,1048.000000,1.0,2,13,0
3744,21,2099,C,NaN,NaN,1.0,2,13,0
3745,21,2976,C,2654.0,1141.722222,1.0,2,13,0
3746,21,3256,C,1309.0,1453.777778,1.0,2,8,1


In [58]:
morph_predicted[morph_predicted['Lk_cell'] == 1]

,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters,Lk_cell
0,1,10,A,2559.0,9460.777778,1.0,2,0,1
1,1,11,A,990.0,3695.222222,1.0,2,3,1
2,1,44,A,619.0,10436.333333,1.0,2,0,1
3,1,65,A,2655.0,6428.111111,1.0,2,9,1
5,1,205,A,2423.0,2864.777778,1.0,2,3,1
...,...,...,...,...,...,...,...,...,...
3730,21,1031,C,1059.0,1499.444444,1.0,2,10,1
3734,21,1286,C,1438.0,1556.666667,1.0,2,8,1
3740,21,1885,C,837.0,1387.555556,1.0,2,10,1
3746,21,3256,C,1309.0,1453.777778,1.0,2,8,1


In [59]:
# save list of all patient's CD34+ cells
pos_to_save = morph_predicted[morph_predicted['Lk_cell'] == 1][['FOV', 'cell_ID']]
pos_to_save['Patient'] = patient

### Visualize all cells with cluster names, highlight likely Lk cells

In [60]:
# create list of fovs
fovs = list(set(morph_predicted['FOV'].tolist()))
fovs.sort()
    
for fov in fovs:
    
    # subset FOV        
    one_fov = morph_predicted[morph_predicted['FOV'] == fov]
    
    # Load img
    location = img_dirs[patient]
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            img = cv2.imread(location + item)
            print(item)
            break
            
    # brighten image
    # define the alpha and beta
    alpha = 1.5 # Contrast control
    beta = 35 # Brightness control

    # call convertScaleAbs function
    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    # Load masks
    location = mask_dir
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break

    # get all mask values
    fov_cells = np.unique(labels)
    
    # Load centroids
    coords_dir = mask_dir + 'cell_centroids/'
    dirlist = listdir(coords_dir)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
            
    # merge prediction and centroid tables
    one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)
    one_fov = one_fov.rename(columns={"cell_ID": "label"})
                
    combo = pd.merge(one_fov,centroids,left_on='label',right_on='label',how='left')
    
    # visualize all segmented cells
    boundaries = segmentation.find_boundaries(labels, connectivity=1, mode='inner', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [255,255,0]
    
    # create list of CD34+ cells
    pos_cells = combo[combo['Lk_cell'] == 1]['label'].tolist()
    pos_cells = [int(i) for i in pos_cells]
    # set all labels of other clusters to zero
    labels_edited = labels.copy()
    for cell in fov_cells:
        if cell in pos_cells:
            pass
        else:
            labels_edited[labels_edited == cell] = 0

    # add colored outline
    boundaries = segmentation.find_boundaries(labels_edited, connectivity=1, mode='thick', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [0,0,255]

    # get unique phenograph clusters
    clusters = list(set(combo['PhenoGraph_clusters'].tolist()))
    
    for cluster in clusters:

        # create list of this cluster's cells
        pos_cells_df = combo[combo['PhenoGraph_clusters'] == cluster]

        for k in range(pos_cells_df.shape[0]):
            img = cv2.putText(img, str(cluster), (int(pos_cells_df['x'].iloc[k]), int(pos_cells_df['y'].iloc[k])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255,255,255], 2, cv2.LINE_AA)
            
    # save image
    cv2.imwrite('CD34_subclusters_overlaid/' + patient + '_FOV' + str(fov).zfill(2) + '_CD34_subcluster.png', img)

20220228_110038_S3_C902_P99_N99_F001_normalized.png
20220228_110038_S3_C902_P99_N99_F001_normalized_cp_masks.tif


/tmp/ipykernel_2140715/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220228_110038_S3_C902_P99_N99_F002_normalized.png
20220228_110038_S3_C902_P99_N99_F002_normalized_cp_masks.tif


/tmp/ipykernel_2140715/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220228_110038_S3_C902_P99_N99_F003_normalized.png
20220228_110038_S3_C902_P99_N99_F003_normalized_cp_masks.tif


...

/tmp/ipykernel_2140715/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220228_110038_S3_C902_P99_N99_F020_normalized.png
20220228_110038_S3_C902_P99_N99_F020_normalized_cp_masks.tif


/tmp/ipykernel_2140715/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220228_110038_S3_C902_P99_N99_F021_normalized.png
20220228_110038_S3_C902_P99_N99_F021_normalized_cp_masks.tif


/tmp/ipykernel_2140715/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


In [61]:
# save list of CD34+ lk cells
pos_to_save.to_csv(patient + '_LkCells.csv', index=False)